In [5]:
import fitz  # PyMuPDF
import csv
import re

# 1. 讀取 PDF 中的文字
pdf_file = "JK37415119.pdf"
doc = fitz.open(pdf_file)

invoice_data = {
    "發票號碼": "",
    "發票日期": "",
    "品名": [],
    "數量": [],
    "單價": [],
    "金額": [],
    "備註": [],
    "銷售額合計": 0,
    "總計": 0
}

for page_num in range(len(doc)):
    page = doc[page_num]
    text = page.get_text()

    # 2. 提取發票號碼和發票日期
    if not invoice_data["發票號碼"]:
        match_invoice = re.search(r'發票號碼:(\w+)', text)
        if match_invoice:
            invoice_data["發票號碼"] = match_invoice.group(1)
    
    if not invoice_data["發票日期"]:
        match_date = re.search(r'(\d{4}-\d{2}-\d{2})', text)
        if match_date:
            invoice_data["發票日期"] = match_date.group(1)

    # 3. 提取品名、數量、單價、金額等資訊
    lines = text.split("\n")
    for line in lines:
        # 簡單判斷可能是品項資訊的行
        item_match = re.match(r'(.*)\s+(\d+)\s+([\d,.]+)\s+([\d,.]+)', line)
        if item_match:
            item_name, quantity, unit_price, amount = item_match.groups()
            invoice_data["品名"].append(item_name.strip())
            invoice_data["數量"].append(quantity.strip())
            invoice_data["單價"].append(unit_price.strip())
            invoice_data["金額"].append(amount.strip())
            invoice_data["備註"].append("")  # 如果備註有額外資訊，可擴展解析邏輯

    # 4. 提取銷售額合計與總計
    match_sales_total = re.search(r'銷售額合計\s+([\d,]+)', text)
    match_grand_total = re.search(r'總計\s+([\d,]+)', text)

    if match_sales_total:
        invoice_data["銷售額合計"] = match_sales_total.group(1).replace(",", "")
    if match_grand_total:
        invoice_data["總計"] = match_grand_total.group(1).replace(",", "")

# 5. 將資料寫入 CSV 檔案
output_file = "invoice_data_with_date.csv"
with open(output_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    # 寫入標題
    writer.writerow(["發票號碼", "發票日期", "品名", "數量", "單價", "金額", "備註", "銷售額合計", "總計"])
    
    
    # 寫入品項資料，每行加上日期、銷售額合計與總計
    for i in range(len(invoice_data["品名"])):
        print(i)
        writer.writerow([
            invoice_data["發票號碼"],
            invoice_data["發票日期"],
            invoice_data["品名"][i],
            invoice_data["數量"][i],
            invoice_data["單價"][i],
            invoice_data["金額"][i],
            invoice_data["備註"][i],
            invoice_data["銷售額合計"],
            invoice_data["總計"]
        ])

print(f"發票資料已存成 CSV 檔案：{output_file}")


發票資料已存成 CSV 檔案：invoice_data_with_date.csv
